In [290]:
import pvlib
import pandas as pd

from pvlib.temperature import TEMPERATURE_MODEL_PARAMETERS

temperature_model_parameters = TEMPERATURE_MODEL_PARAMETERS['sapm']['open_rack_glass_glass']

# load some module and inverter specifications
cec_modules = pvlib.pvsystem.retrieve_sam('cecmod')


cec_inverters = pvlib.pvsystem.retrieve_sam('cecinverter')

# cec_module = cec_modules['Sunpreme_Inc__SNPM_GxB_510']
# cec_df=pd.read_excel('PVLibCEC.xlsx')

cec_inverter = cec_inverters['ABB__MICRO_0_25_I_OUTD_US_208__208V_']
# cec_module.to_dict()
# cec_df
pvsyst_module_map = {
    'Technology': 'Mono-c-Si',
    'Bifacial': 1,
    'STC': 550.62,
    'PTC': 516.4,
    'A_c': 2.52,
    'Length': None,  # NaN is represented as None in Python
    'Width': None,   # NaN is represented as None in Python
    'N_s': 72,
    'I_sc_ref': 14,
    'V_oc_ref': 49.9,
    'I_mp_ref': 13.11,
    'V_mp_ref': 42,
    'alpha_sc': 0.00546,
    'beta_oc': -0.13024,
    'T_NOCT': 44.8,
    'a_ref': 1.82357,
    'I_L_ref': 14.0167,
    'I_o_ref': 1.78e-11,
    'R_s': 0.164846,
    'R_sh_ref': 138.077,
    'Adjust': 6.23131,
    'gamma_r': -0.33,
    'BIPV': 'N',
    'Version': '2023.10.31',
    'Date': '11/16/2022',
    'Name': 'JA Solar JAM72D30-550/MB'
}
pvsyst_module_series=pd.Series(pvsyst_module_map)
cec_inverter



Vac                          208
Pso                     2.089607
Paco                       250.0
Pdco                  259.588593
Vdco                        40.0
C0                     -0.000041
C1                     -0.000091
C2                      0.000494
C3                     -0.013171
Pnt                        0.075
Vdcmax                      50.0
Idcmax                  6.489715
Mppt_low                    30.0
Mppt_high                   50.0
CEC_Date                     NaN
CEC_Type     Utility Interactive
Name: ABB__MICRO_0_25_I_OUTD_US_208__208V_, dtype: object

In [291]:
DEFAULT_INVERTER = {
    'Vac': 277,  # Nominal AC voltage (V)
    'Pso': 28.358692,  # Power consumption during operation (W)
    'Paco': 2500,  # Maximum AC power output (W)
    'Pdco': 2592.4729,  # Maximum DC power output (W)
    'Vdco': 360.0,  # Nominal DC voltage (V)
    'C0': -0.000008,  # Curvature between AC power and DC power (1/W)
    'C1': -0.000045,  # Coefficient of `Pdco` variation with DC input voltage (1/V)
    'C2': 0.00041,  # Coefficient of inverter power consumption loss variation with DC input voltage (1/V)
    'C3': -0.002524,  # Coefficient of C0 variation with DC input voltage (1/V)
    'Pnt': 0.5,  # Inverter night time loss (kW)
    'Vdcmax': 416.0,  # Maximum DC voltage (V)
    'Idcmax': 7.201314,  # Maximum DC current (A)
    'Mppt_low': 100.0,  # Minimum MPPT DC voltage (V)
    'Mppt_high': 416.0,  # Maximum MPPT DC voltage (V)
    'Name': 'ABB__UNO_2_5_I_OUTD_S_US__277V_',  # Name of the inverter
    'efficiency':0.98
}

DEFAULT_MODULE={
'Technology': 'Thin Film',
 'Bifacial': 1,
 'STC': 509.97,
 'PTC': 479.6,
 'A_c': 2.591,
 'Length': 1.981,
 'Width': 1.308,
 'N_s': 96,
 'I_sc_ref': 9.4,
 'V_oc_ref': 74.7,
 'I_mp_ref': 8.9,
 'V_mp_ref': 57.3,
 'alpha_sc': 0.00094,
 'beta_oc': -0.19422,
 'T_NOCT': 45.5,
 'a_ref': 2.41017,
 'I_L_ref': 9.40894,
 'I_o_ref': 3.237911e-13,
 'R_s': 1.135045,
 'R_sh_ref': 1193.327026,
 'Adjust': -20.561962,
 'gamma_r': -0.3,
 'BIPV': 'N',
 'Version': 'SAM 2018.11.11 r2',
 'Date': '1/3/2019'
 }
custom_module=pd.Series(DEFAULT_MODULE)
custom_inverter=pd.Series(DEFAULT_INVERTER)
custom_inverter


Vac                                       277
Pso                                 28.358692
Paco                                     2500
Pdco                                2592.4729
Vdco                                    360.0
C0                                  -0.000008
C1                                  -0.000045
C2                                    0.00041
C3                                  -0.002524
Pnt                                       0.5
Vdcmax                                  416.0
Idcmax                               7.201314
Mppt_low                                100.0
Mppt_high                               416.0
Name          ABB__UNO_2_5_I_OUTD_S_US__277V_
efficiency                               0.98
dtype: object

In [292]:
from pvlib.location import Location
from pvlib.pvsystem import PVSystem, FixedMount,Array
location = Location(latitude=32.2, longitude=-110.9)

arrs=[]
mount=FixedMount(surface_tilt=45,
                 surface_azimuth=180,
                 racking_model='open_rack',
                 module_height=1)

# for _ in range(4):
#     newArr=Array(mount=mount,
#                  albedo=0.25,
#                  module_parameters=pvsyst_module_series,
#                  module_type='glass_glass',
#                  modules_per_string=25,
#                  strings=3)
#     arrs.append(newArr)

# system=PVSystem(arrays=arrs,inverter_parameters=custom_inverter)
system = PVSystem(surface_tilt=45, surface_azimuth=180,
                  module_parameters=pvsyst_module_map,
                  inverter_parameters=DEFAULT_INVERTER,
                  strings_per_inverter=2,
                  modules_per_string=25,
                  racking_model='open_rack',
                  module_type='glass_glass',
                  albedo=0.25)

system

PVSystem:
  name: None
  Array:
    name: None
    mount: FixedMount(surface_tilt=45, surface_azimuth=180, racking_model='open_rack', module_height=None)
    module: None
    albedo: 0.25
    module_type: glass_glass
    temperature_model_parameters: {'a': -3.47, 'b': -0.0594, 'deltaT': 3}
    strings: 2
    modules_per_string: 25
  inverter: None

In [293]:
import pandas as pd
water=pvlib.atmosphere.gueymard94_pw(9.02, 74)
print(f'water= {water}')
weather_data={
    'ghi':453,
    'dni':917,
    'dhi':50,
    'temp_air':9.02,
    'wind_speed':5.66,
    'precipitable_water':water
}
weather = pd.DataFrame([weather_data],
                       index=[pd.Timestamp('20170401 1200', tz='US/Arizona')])
weather


water= 1.420983331355153


,ghi,dni,dhi,temp_air,wind_speed,precipitable_water
2017-04-01 12:00:00-07:00,453,917,50,9.02,5.66,1.420983


In [294]:
from pvlib.modelchain import ModelChain
mc=ModelChain(system=system,location=location,aoi_model='no_loss')
mc


ModelChain: 
  name: None
  clearsky_model: ineichen
  transposition_model: haydavies
  solar_position_method: nrel_numpy
  airmass_model: kastenyoung1989
  dc_model: cec
  ac_model: sandia_inverter
  aoi_model: no_aoi_loss
  spectral_model: first_solar_spectral_loss
  temperature_model: sapm_temp
  losses_model: no_extra_losses

In [295]:
mc.run_model(weather)


ModelChain: 
  name: None
  clearsky_model: ineichen
  transposition_model: haydavies
  solar_position_method: nrel_numpy
  airmass_model: kastenyoung1989
  dc_model: cec
  ac_model: sandia_inverter
  aoi_model: no_aoi_loss
  spectral_model: first_solar_spectral_loss
  temperature_model: sapm_temp
  losses_model: no_extra_losses

In [296]:
# mc.results.dc.reset_index().loc[0].to_dict()
mc.results.dc

,i_sc,v_oc,i_mp,v_mp,p_mp,i_x,i_xx
2017-04-01 12:00:00-07:00,25.758395,1217.273488,24.084435,1022.709502,24631.380398,25.435102,18.179092


In [297]:
mc.results.ac

2017-04-01 12:00:00-07:00    2500.0
dtype: float64

In [287]:
sandia_modules=pvlib.pvsystem.retrieve_sam('SandiaMod')
module = sandia_modules['BP_Solar_BP3220N_Module___2010_'] #18% eff, c-Si
inv = cec_inverters['ABB__MICRO_0_3_I_OUTD_US_208__208V_']
system = PVSystem(module_parameters=module,inverter_parameters=inv,surface_azimuth=180,
                    modules_per_string=23,strings_per_inverter=220,
                    racking_model='open_rack',
                  module_type='glass_glass')
mc2=ModelChain(system=system,location=location)

In [288]:
mc2.run_model(weather=weather)

ModelChain: 
  name: None
  clearsky_model: ineichen
  transposition_model: haydavies
  solar_position_method: nrel_numpy
  airmass_model: kastenyoung1989
  dc_model: sapm
  ac_model: sandia_inverter
  aoi_model: sapm_aoi_loss
  spectral_model: sapm_spectral_loss
  temperature_model: sapm_temp
  losses_model: no_extra_losses

In [289]:
mc2.results.ac

2017-04-01 12:00:00-07:00    300.0
dtype: float64